# Example of reading Device DNA with PYNQ API

In [1]:
# Load packages we need
from pprint import pprint
from pynq import Overlay, DefaultIP

## Load the overlay and see what's inside

In [2]:
ol = Overlay("/home/xilinx/DNA_extractor.bit")
ol.download()

pprint(ol.ip_dict)  # to see what's inside

{'zynq_AXI_DNA_0': {'addr_range': 4096,
                    'driver': <class 'pynq.overlay.DefaultIP'>,
                    'fullpath': 'zynq_AXI_DNA_0',
                    'gpio': {},
                    'interrupts': {},
                    'phys_addr': 1136656384,
                    'state': None,
                    'type': 'ABR:user:zynq_AXI_DNA:1.0'}}


## Create the memory map

We will use the PYNQ API to create an easy to use memory map on the PS side.

In [3]:
class MemoryMap(DefaultIP):
    def __init__(self, description):
        super().__init__(description)

    bindto = ['ABR:user:zynq_AXI_DNA:1.0']
    # found in ip_dict above as 'type'

    # Creating 'getter' for mmio registers
    @property
    def reg0(self):
        return self.read(0x00)
        
    @property
    def reg1(self):
        return self.read(0x04)
    
    @property
    def reg2(self):
        return self.read(0x08)
        
    @property
    def reg3(self):
        return self.read(0x0C)

## Reload overlay to bind to our new MMIO driver

In [4]:
ol = Overlay("/home/xilinx/DNA_extractor.bit")
ol.download()

pprint(ol.ip_dict)  # to see what's inside

{'zynq_AXI_DNA_0': {'addr_range': 4096,
                    'driver': <class '__main__.MemoryMap'>,
                    'fullpath': 'zynq_AXI_DNA_0',
                    'gpio': {},
                    'interrupts': {},
                    'phys_addr': 1136656384,
                    'state': None,
                    'type': 'ABR:user:zynq_AXI_DNA:1.0'}}


## Test it out

We read two 32b registers in order to capture the 57b Device DNA, the additional 7 bits are tied to zero.

In [5]:
DNA0 = ol.zynq_AXI_DNA_0.reg0
DNA1 = ol.zynq_AXI_DNA_0.reg1
print("DNA: %#0.8x%0.8x" %(DNA0, DNA1))

DNA: 0x0054235884625018


In [6]:
# These registers should be zeros, check for peace of mind
DNA2 = ol.zynq_AXI_DNA_0.reg2
DNA3 = ol.zynq_AXI_DNA_0.reg3
print("zeros: %#0.8x%0.8x" %(DNA2, DNA3))

zeros: 0x0000000000000000
